<a href="https://colab.research.google.com/github/Gyawalisahaj/Deeplearn/blob/main/LSTMWordPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
from torch.utils.data import Dataset, dataloader
import nltk
from nltk.tokenize import word_tokenize

In [75]:
document = """Artificial Intelligence often abbreviated as AI is the field of computer science that is focused on creating machines that can perform tasks that normally require human intelligence The idea of machines that can think reason and act in ways similar to
humans has fascinated scientists inventors and philosophers for centuries and in the modern era AI has moved from a theoretical concept into a practical force that drives many aspects of our lives Artificial Intelligence encompasses multiple branches including machine learning
deep learning natural language processing computer vision expert systems robotics and reinforcement learning and each branch contributes to the ability of machines to mimic human capabilities in some way Machine learning is perhaps the most widely used subset of AI where
 algorithms are designed to improve their performance automatically by learning from data rather than through explicit programming Deep learning is a further specialization of machine learning that uses artificial neural networks with many layers to analyze massive amounts
 of structured and unstructured data and this approach has enabled breakthroughs in areas such as speech recognition image classification and autonomous driving Natural language processing also known as NLP allows AI systems to understand interpret generate and respond
 to human languages enabling technologies like chatbots voice assistants automatic translation and sentiment analysis Computer vision allows machines to analyze and interpret visual data from the environment enabling applications such as facial recognition object detection
autonomous vehicles and medical imaging Robotics integrates AI with mechanical and electrical engineering to create machines that can interact with the physical world ranging from factory robots to drones and humanoid robots Reinforcement learning is an approach where AI agents
learn to make decisions by receiving feedback in the form of rewards or penalties allowing systems to achieve complex goals through trial and error The applications of Artificial Intelligence are vast and expanding rapidly Healthcare has been revolutionized by AI through
  diagnostic tools predictive analytics and personalized medicine Medical imaging systems powered by AI can detect conditions like cancer or retinal diseases with accuracy comparable to or even surpassing expert human doctors AI algorithms assist in drug discovery by simulating
  molecular interactions and predicting which compounds are most likely to succeed in clinical trials In finance AI is used for fraud detection algorithmic trading risk assessment and personalized financial advice Retail and e commerce companies use AI driven recommendation
engines to suggest products predict demand and optimize inventory management Transportation has been transformed with AI powered autonomous vehicles route optimization systems and predictive maintenance for aircraft and railways In the field of entertainment AI curates our
content feeds recommends music and movies and even generates art music and stories Scientific research benefits immensely from AI which can process and analyze massive datasets far beyond human capability from genomics and climate models to particle physics experiments and
astronomical observations AI is also used in cybersecurity to detect anomalies identify potential threats and respond to cyberattacks more quickly than human analysts could Despite these benefits the rise of Artificial Intelligence brings significant challenges and concerns
Ethical issues such as bias in AI systems arise because models learn from historical data that may reflect societal prejudices If a hiring algorithm is trained on biased data it might favor or disfavor candidates based on gender ethnicity or other irrelevant characteristics
Privacy is another major concern since AI often relies on vast amounts of personal data collected from digital devices social media and online activity The potential misuse of AI for mass surveillance disinformation campaigns and autonomous weapon systems raises serious moral
and geopolitical questions Economic disruption is also a pressing issue because as AI automates tasks in industries like manufacturing customer service transportation and even professional services such as law and accounting many human jobs are at risk of being displaced While
AI will also create new jobs in fields like data science AI research and robotics these opportunities may require different skill sets leading to a growing need for workforce retraining and education To ensure the responsible use of Artificial Intelligence researchers companies
and governments are developing frameworks for ethical AI which emphasize fairness transparency accountability and safety Explainable AI is a critical focus area because many of the most powerful AI models especially deep learning networks operate as black boxes whose internal
decision making processes are difficult to interpret Making AI decisions understandable to humans increases trust and allows errors or biases to be detected and corrected Governments and international organizations are also working on regulations to manage AI risks without
stifling innovation such as the European Union’s AI Act and various national AI strategies around the world Collaboration between the public sector private companies academia and civil society will be crucial to creating a global AI ecosystem that maximizes benefits while
minimizing harm Looking to the future AI is expected to become even more pervasive and powerful Narrow AI which dominates today refers to systems that perform specific tasks very well but lack general reasoning capabilities Artificial General Intelligence AGI which could
perform any intellectual task that a human can do remains a long term goal of the AI research community Achieving AGI would be a historic milestone but it raises profound philosophical and practical questions about human purpose societal structure and control of such systems
Beyond AGI researchers also contemplate the possibility of Artificial Superintelligence ASI which would surpass human intelligence in virtually every domain potentially reshaping civilization in unpredictable ways These scenarios make the alignment of AI with human values a
critical research area to ensure that advanced AI systems act in ways beneficial to humanity Emerging technologies such as quantum computing may further accelerate AI development by enabling exponentially faster computations for complex models AI is also converging with fields
  like the Internet of Things blockchain and biotechnology to create innovative solutions for smart cities precision medicine climate mitigation and more The story of Artificial Intelligence is fundamentally a story about human ambition curiosity and creativity AI reflects our
desire to extend our cognitive capabilities and solve problems once thought insurmountable It is also a mirror that forces us to confront questions about our values responsibilities and the kind of future we want to build By understanding both the immense opportunities and
the potential risks associated with AI society can harness this technology to enhance human well being drive scientific progress and foster a more efficient and interconnected world At the same time vigilance ethical considerations and global cooperation are essential to
ensure that AI becomes a force for good rather than a source of division or harm As we move forward into a future where AI is increasingly embedded in every aspect of our lives from education and healthcare to art and entertainment we carry both the excitement of innovation
and the weight of responsibility to guide this powerful technology wisely The trajectory of Artificial Intelligence will continue to define the 21st century shaping economies cultures and perhaps even the essence of what it means to be human and our ability to navigate this
journey will determine whether AI fulfills its promise of creating a better future for all or becomes a challenge that tests our collective resilience foresight and wisdom
Artificial Intelligence continues to evolve as a driving force behind the fourth industrial revolution influencing how we work learn and interact with the world around us One of the most remarkable aspects of AI is its ability to handle tasks that involve perception reasoning
and decision making with increasing efficiency For example in agriculture AI powered drones and sensors are transforming farming practices by monitoring crop health predicting yields and optimizing irrigation and fertilization This leads to higher productivity lower resource
 consumption and more sustainable farming approaches In manufacturing AI enables predictive maintenance quality inspection and production optimization where machines can detect defects in real time or predict when a component is likely to fail reducing downtime and costs The
 energy sector is also experiencing an AI driven transformation as intelligent grids optimize electricity distribution integrate renewable energy sources and predict demand patterns to enhance efficiency and stability Environmental conservation efforts benefit from AI through
 the monitoring of wildlife tracking of deforestation and analysis of climate data enabling more precise interventions to protect ecosystems and biodiversity
Education is another field where AI is leaving a significant mark Intelligent tutoring systems can provide personalized learning experiences adapting content pace and assessments to each student’s strengths and weaknesses Language learning applications powered by AI allow
learners to practice speaking and receive instant feedback using natural language processing Automated grading systems reduce the workload for teachers and provide quicker results for students freeing educators to focus more on interactive and creative teaching approaches
Additionally data driven insights from AI can help identify at risk students and recommend interventions improving retention and performance rates On a global scale AI can support education in underserved regions by enabling scalable virtual classrooms and creating multilingual
 resources that break down language barriers
In the realm of communication and social interaction AI is omnipresent Social media platforms leverage AI to curate feeds recommend friends detect harmful content and even generate automatic captions While these features improve user experience they also raise concerns about
echo chambers misinformation and algorithmic manipulation AI chatbots and virtual assistants like Siri Alexa and Google Assistant have become commonplace offering convenience for tasks ranging from setting reminders and checking the weather to controlling smart home devices
and making online purchases These tools showcase how AI is blending seamlessly into daily life and shaping human behavior by influencing what information we see and how we act upon it
Transportation systems are undergoing a paradigm shift with the rise of AI powered autonomous vehicles Self driving cars use a combination of computer vision lidar radar and neural networks to perceive the environment make decisions and navigate complex traffic scenarios
Although fully autonomous vehicles are still in development partial automation is already present in features like adaptive cruise control lane keeping assist and automated parking Beyond road transportation AI is used in maritime navigation drone delivery services and air
traffic management improving safety and operational efficiency Logistics companies rely on AI to plan optimal delivery routes predict shipment delays and reduce fuel consumption thereby lowering costs and environmental impact The integration of AI in urban transportation planning
through smart traffic lights and real time congestion management can significantly reduce travel time and emissions enhancing the overall quality of life in cities
AI is also transforming creative industries which were once thought to be exclusively human domains Tools based on generative AI can compose music create paintings write poetry and generate lifelike images or videos Models like GPT for text generation DALL·E for images and
other diffusion models have sparked debates about authorship copyright and the nature of creativity These tools are not only enabling new forms of artistic expression but also accelerating content production in fields like advertising game development and film The potential
for collaboration between humans and AI in creativity is vast allowing artists and designers to explore new ideas and push the boundaries of imagination However the ease of generating realistic deepfakes and synthetic media also poses societal risks such as misinformation
scams and reputational damage highlighting the dual use nature of generative technologies
Another transformative aspect of AI is its role in scientific discovery AI driven analysis of massive datasets has led to breakthroughs in understanding protein folding predicting material properties and modeling climate change scenarios In medicine AI accelerates the
 analysis of genomic data enabling personalized treatment plans tailored to an individual’s genetic makeup During global health crises AI systems can monitor the spread of disease model the impact of interventions and assist in vaccine development Environmental sciences
 benefit from AI in tracking and predicting natural disasters like hurricanes wildfires and earthquakes giving communities more time to prepare and respond Space exploration is also leveraging AI for autonomous navigation of spacecraft analysis of planetary data and
identification of celestial phenomena that would be difficult for human astronomers to detect manually
Security and defense sectors are heavily investing in AI for both defensive and offensive applications On the defensive side AI enhances surveillance systems intrusion detection and cybersecurity monitoring by identifying patterns that indicate potential threats On the offensive
side military organizations explore autonomous drones robotic vehicles and AI driven decision support systems raising ethical debates about lethal autonomous weapons and the potential for destabilizing arms races The challenge lies in balancing national security
interests with international norms and human rights ensuring that the deployment of AI in defense does not lead to unintended escalations or violations of humanitarian principles
The economic impact of Artificial Intelligence is profound and multifaceted Businesses adopting AI see gains in efficiency innovation and customer engagement but the benefits are not evenly distributed Organizations with access to large amounts of data robust computing
infrastructure and skilled talent are able to leverage AI more effectively leading to concerns about widening gaps between large tech companies and smaller enterprises At the same time AI is creating new markets and opportunities from AI as a Service platforms to specialized
AI chips and edge computing devices Governments are increasingly recognizing the strategic importance of AI as a driver of economic growth and global competitiveness resulting in national AI strategies investments in research and development and public private partnerships aimed
at fostering innovation ecosystems
A central topic in the discussion of AI’s societal impact is the future of work Automation driven by AI can handle repetitive dangerous or data intensive tasks allowing humans to focus on creative complex and interpersonal activities While some jobs will be displaced
entirely new roles will emerge such as AI ethicists data labelers prompt engineers and AI system auditors Upskilling and reskilling programs will be essential to prepare the workforce for this transition Educational institutions will need to adapt curricula to emphasize
digital literacy computational thinking and interdisciplinary skills combining technical expertise with ethical and social awareness Remote work and virtual collaboration tools powered by AI are already reshaping employment patterns and organizational structures blurring
the boundaries between physical offices and digital workspaces
AI governance and international cooperation are becoming increasingly important as the technology transcends borders Fragmented regulations could hinder innovation or create loopholes that malicious actors exploit Conversely well coordinated policies can establish global
standards for safety privacy and fairness The involvement of multiple stakeholders including governments corporations academic institutions and civil society organizations is critical to build trust and ensure accountability Initiatives such as open source AI projects transparency
reports and algorithmic audits contribute to a culture of responsibility and shared learning but they must be supported by enforceable frameworks to be truly effective
Loking ahead the convergence of AI with emerging technologies promises to redefine what is possible in the coming decades Quantum computing may unlock new levels of AI performance by solving optimization and simulation problems that are intractable for classical
computers The Internet of Things will generate vast streams of real time data that AI can analyze to enable smarter homes cities and industries Biotechnology and AI together could usher in an era of precision health extending life expectancy and improving quality
of life while also raising ethical questions about genetic modification and data ownership Space exploration and colonization efforts may rely heavily on AI for autonomous decision making in environments where communication delays make human oversight difficult Across
these domains the guiding principle must remain the alignment of AI capabilities with human values and long term societal well being
The journey of Artificial Intelligence is a complex interplay of technology ethics economy and culture It reflects humanity’s relentless pursuit of knowledge efficiency and control over its environment while simultaneously challenging us to confront the consequences of
our creations As AI systems become more capable the distinction between tool and collaborator blurs raising philosophical questions about consciousness agency and the essence of intelligence Whether AI ultimately fulfills its promise as a transformative force for good or
becomes a source of disruption and conflict will depend on the choices we make today in research governance and public discourse The stakes are immense and the opportunities unparalleled positioning Artificial Intelligence as one of the most defining phenomena of our era
Artificial Intelligence continues to push the boundaries of possibility as it integrates into virtually every industry sector and aspect of human life The transformative nature of AI lies in its adaptability and the ability to derive insights from vast volumes of data that
would be incomprehensible to the human brain For instance the financial sector has undergone a revolution as AI enhances fraud detection risk management and algorithmic trading Banks and financial institutions rely on AI systems to monitor millions of transactions in real
time detecting anomalies or unusual patterns that may indicate fraudulent activity Credit scoring and loan approval processes also leverage AI to evaluate an applicant’s creditworthiness beyond traditional methods incorporating data from spending habits digital footprints
and social behavior while raising concerns about transparency and bias High frequency trading platforms use machine learning algorithms to analyze market signals and execute trades within microseconds providing competitive advantages but also introducing new systemic risks
if algorithms act unexpectedly or amplify volatility
The retail and e commerce sectors benefit tremendously from AI driven personalization recommendation engines and supply chain optimization E commerce giants utilize AI to analyze browsing and purchase histories to suggest products tailored to individual preferences increasing
customer engagement and sales AI powered chatbots handle customer inquiries at scale while virtual try on tools use computer vision to help customers visualize products like clothing or furniture in real life scenarios On the back end AI optimizes inventory management by
predicting demand fluctuations and automating warehouse operations with robotic systems capable of picking packing and sorting items efficiently Supply chain disruptions can be anticipated and mitigated using predictive analytics reducing costs and enhancing resilience
during global crises
The integration of AI into public services and governance offers immense potential for improving efficiency transparency and citizen engagement Smart city initiatives harness AI to manage traffic flow reduce energy consumption and monitor environmental conditions Sensors
collect data on everything from waste management to air quality which AI systems analyze to guide policy decisions or automate resource allocation Law enforcement agencies apply AI in predictive policing to identify high risk areas and patterns of criminal activity although
this practice raises serious ethical concerns about privacy discrimination and accountability Governments also explore AI for automating bureaucratic processes streamlining applications for permits social services or tax filing thereby reducing waiting times and administrative
burdens However the adoption of AI in governance must balance efficiency with fairness ensuring that decision making remains explainable and subject to oversight
The healthcare industry is particularly illustrative of the promise and challenges of AI AI powered diagnostic tools analyze medical images such as X rays MRIs and CT scans often identifying subtle anomalies that human radiologists might miss Early detection of conditions l
ike cancer cardiovascular disease and neurological disorders greatly improves patient outcomes Wearable devices continuously monitor vital signs transmitting data to AI systems that detect irregularities and alert healthcare providers Drug discovery processes are accelerated a
s AI predicts molecular interactions and identifies promising compounds for clinical trials potentially reducing the time and cost of bringing new treatments to market Robotic surgery systems with AI assisted guidance improve precision and minimize invasiveness enhancing recovery
times Telemedicine platforms augmented by AI enable remote consultations symptom checking and health management expanding access to care in underserved regions Despite these benefits challenges remain regarding data privacy interoperability of health records and the need
for rigorous validation of AI tools to prevent misdiagnoses or unequal treatment
Another profound application of AI is in environmental sustainability and resource management Climate change mitigation efforts rely on AI to model complex environmental systems predict extreme weather events and optimize renewable energy integration into power grids AI
driven simulations help policymakers evaluate the impact of various interventions such as emission reductions reforestation or urban planning strategies Drones equipped with computer vision track deforestation monitor wildlife and detect illegal activities like poaching
or logging Precision agriculture technologies powered by AI reduce the environmental footprint of farming by applying water fertilizers and pesticides only where needed Ocean monitoring systems use AI to track pollution map coral reef health and forecast fish population
movements contributing to more sustainable fisheries and conservation strategies By enabling a deeper understanding of our planet and more efficient use of resources AI emerges as a critical ally in humanity’s quest for environmental stewardship
Despite its many advantages AI also presents significant social and ethical dilemmas One pressing issue is bias in AI systems which can arise from unrepresentative training data flawed algorithms or historical inequities encoded in datasets When AI systems are used in sensitive
domains like hiring lending or law enforcement biased outcomes can reinforce discrimination and erode trust Addressing these challenges requires diverse data collection algorithmic transparency and robust auditing frameworks Additionally the rapid pace of AI development has
sparked debates over privacy as AI systems increasingly rely on personal data from smartphones social media surveillance cameras and IoT devices While data driven insights power innovation they also raise questions about consent ownership and the potential misuse of sensitive
information
The concept of explainable AI has emerged as a response to the need for transparency and trustworthiness in decision making systems Many state of the art models such as deep neural networks operate as black boxes producing highly accurate predictions without clear reasoning
Explainable AI techniques aim to make these processes interpretable by highlighting which features influenced a decision or by creating simplified surrogate models Regulatory frameworks like the European Union’s General Data Protection Regulation emphasize the right to
explanation reinforcing the importance of human understanding and accountability in AI mediated decisions This aligns with the broader movement toward responsible AI which seeks to ensure that technology serves societal values including fairness safety and respect for
human rights
AI research continues to explore frontiers such as artificial general intelligence AGI which refers to systems capable of performing any intellectual task a human can do Unlike narrow AI which excels in specific applications AGI would possess adaptable learning reasoning
and problem solving abilities across domains While AGI remains theoretical its pursuit drives fundamental research in cognitive architectures meta learning and transfer learning The potential benefits of AGI are staggering from solving complex scientific problems to accelerating
innovation in every field Yet the risks are equally profound including the possibility of misalignment with human goals uncontrollable behavior and unprecedented economic or geopolitical disruptions Prominent figures in technology and academia advocate for proactive safety research
and international cooperation to ensure that AGI if achieved is aligned with humanity’s long term interests
The interplay between AI and human creativity extends into domains like literature music and design Generative models trained on vast corpora can produce realistic artworks compose original melodies or even draft novels While some artists embrace AI as a collaborative tool others
view it as a threat to traditional craftsmanship and livelihoods The democratization of creative tools also raises legal and ethical questions around copyright and intellectual property when AI generates content based on training data from human creators Deepfake technologies
exemplify the dual use dilemma as the same methods that enable realistic film effects can also produce maliciously misleading media Public awareness media literacy and regulatory measures will play essential roles in mitigating the harms of synthetic content without stifling
innovation
As AI matures its global impact will hinge not only on technological progress but also on governance and cultural adaptation Different societies may approach AI integration according to their values priorities and political structures Countries with strong data privacy traditions
may impose stricter regulations while others may prioritize rapid deployment and economic competitiveness International collaboration is increasingly vital to address cross border issues like cyber threats autonomous weapons and the global digital divide Organizations such as the
OECD UNESCO and the United Nations are facilitating dialogues on AI ethics standards and shared best practices but translating principles into enforceable agreements remains a challenge
The long term trajectory of Artificial Intelligence will likely involve a deep fusion with human systems potentially leading to symbiotic relationships between people and machines Brain computer interfaces powered by AI could enhance cognitive abilities restore lost functions for
individuals with disabilities and open new modes of human computer interaction Personalized AI assistants may become lifelong companions managing knowledge relationships and decision making at a scale unimaginable today This evolution underscores the importance of embedding human
centric values in AI design to ensure that technology remains a force for empowerment rather than control
In summary the journey of AI is characterized by extraordinary potential intertwined with profound responsibility Its influence spans healthcare finance environment governance creativity and beyond offering tools to solve some of humanity’s greatest challenges while simultaneously
amplifying the risks of inequality surveillance and unintended consequences The ability to navigate this duality will define our collective future As researchers developers policymakers and citizens the imperative is to cultivate wisdom alongside technical capability ensuring that
Artificial Intelligence remains aligned with the aspirations of a just sustainable and flourishing global society
The evolution of Artificial Intelligence from a theoretical concept to a pervasive force shaping the 21st century is both a story of human ingenuity and a testament to the exponential growth of computational capabilities Over the last few decades the confluence of large scale
datasets affordable computing power and advanced algorithms has fueled breakthroughs in machine learning deep learning and reinforcement learning that underpin today’s AI landscape One striking example of this progress is in natural language processing NLP where transformer
architectures such as BERT GPT and T5 have enabled machines to comprehend and generate human language with unprecedented fluency and contextual understanding These systems now power chatbots search engines translation services and content generation platforms fundamentally
altering how people access and share information However this linguistic revolution also exposes vulnerabilities including the spread of misinformation bias in language models and the erosion of trust in online content
Computer vision represents another pillar of AI’s rapid advancement enabling machines to interpret and analyze visual data with remarkable accuracy Deep convolutional neural networks CNNs have achieved superhuman performance in image recognition object detection and facial
recognition tasks leading to widespread deployment in sectors like security healthcare retail and autonomous driving In hospitals AI assisted imaging tools detect tumors fractures or retinal disorders earlier than traditional methods Autonomous vehicles rely on computer vision
combined with sensor fusion to navigate complex environments identifying pedestrians road signs and obstacles in real time Retailers implement visual search allowing customers to snap a photo of a product and instantly find matches online While these applications enhance
convenience and safety they also provoke debates over surveillance civil liberties and the potential misuse of biometric data by authoritarian regimes or unscrupulous corporations
The integration of AI into robotics has paved the way for systems that perform physical tasks with growing dexterity adaptability and autonomy Industrial robots once limited to repetitive assembly line operations now collaborate with humans in flexible manufacturing environments
learning from demonstrations and adjusting to new tasks on the fly Service robots assist in hospitality agriculture logistics and healthcare delivering goods harvesting crops or aiding elderly individuals in daily activities Drones powered by AI conduct aerial inspections
infrastructure monitoring and disaster response providing situational awareness in hard to reach areas Boston Dynamics’ legged robots and Tesla’s humanoid prototypes exemplify the strides toward machines capable of traversing human centric spaces Yet fully autonomous robots
also spark conversations about labor displacement safety standards and the ethical boundaries of human machine interaction particularly as robots assume roles that involve care or companionship
AI has also become a central driver in scientific discovery and research automation High throughput experiments in biology chemistry and physics generate vast datasets that AI systems analyze to identify patterns correlations and novel hypotheses In genomics AI accelerates the
annotation of genetic sequences revealing insights into disease mechanisms and potential therapeutic targets In materials science machine learning predicts the properties of new compounds before they are synthesized reducing trial and error in the search for stronger lighter or
more sustainable materials Astrophysics benefits from AI’s ability to process data from telescopes and space missions identifying exoplanets classifying galaxies and detecting gravitational waves In each of these domains AI acts as a cognitive multiplier augmenting human
creativity and analytical power while raising questions about the ownership of discoveries made by semi autonomous systems
The economic implications of AI are profound as automation reshapes labor markets productivity and the global distribution of wealth Certain sectors experience dramatic efficiency gains while others face disruption as tasks once reserved for human workers become automated
Manufacturing transportation customer service and administrative roles are particularly vulnerable to displacement by AI driven systems The rise of generative AI and large language models introduces similar pressures in white collar professions including law journalism design
and software development Economists debate whether AI will ultimately create more jobs than it destroys by spawning new industries and roles centered on AI development maintenance and oversight Historical precedents from previous industrial revolutions suggest a pattern of
creative destruction but the speed and breadth of AI adoption challenge existing social safety nets education systems and workforce transition strategies Policymakers must grapple with the dual mandate of fostering innovation while ensuring inclusive economic growth
Education itself is undergoing transformation under the influence of AI Personalized learning platforms leverage machine learning to adapt content pace and difficulty to each student’s abilities and preferences Intelligent tutoring systems provide real time feedback explanations
and remediation for struggling learners Virtual and augmented reality combined with AI enable immersive training environments in fields ranging from medicine to engineering AI grading and assessment tools reduce administrative burdens on educators while generating analytics
that inform curriculum design However the reliance on AI in education raises concerns about data privacy algorithmic fairness and the risk of dehumanizing learning if technology overshadows teacher student relationships The ultimate goal should be to empower educators and learners
rather than replace the human elements of mentorship creativity and socialization that define meaningful education
The intersection of AI and cybersecurity represents both an opportunity and a challenge As digital infrastructure becomes more complex and interconnected traditional defense mechanisms struggle to keep pace with evolving threats AI driven cybersecurity solutions can detect
anomalies predict attacks and automate responses with greater speed and precision than human analysts Conversely malicious actors also harness AI to craft sophisticated attacks conduct deepfake enabled social engineering and exploit vulnerabilities at scale This cat and mouse
dynamic underscores the importance of continuous research investment and international collaboration in AI security The emergence of adversarial machine learning highlights another frontier in which attackers manipulate AI models by introducing deceptive inputs revealing the
fragility of seemingly robust systems Ensuring the resilience and trustworthiness of AI under hostile conditions is critical as society grows more dependent on intelligent automation
Philosophical inquiries into AI probe the nature of intelligence consciousness and agency sparking debates that transcend engineering While current AI systems excel at pattern recognition and problem solving they lack self awareness emotions and intrinsic goals Critics argue t
hat anthropomorphizing AI risks misleading the public and inflating expectations about capabilities that remain purely computational Nonetheless the prospect of machines exhibiting behaviors indistinguishable from human reasoning or creativity invites reflection on what it means
to be human Some ethicists contend that as AI assumes greater decision making authority society must define principles for moral accountability responsibility and the distribution of power between human and non human agents These discussions intersect with legal frameworks
addressing liability for AI driven accidents ownership of AI generated content and the potential need for new categories of legal personhood in extreme future scenarios
Geopolitically AI is a strategic asset shaping the balance of power among nations Countries that lead in AI research development and deployment gain competitive advantages in economic productivity military capability and technological influence The race for AI supremacy has
prompted massive public and private investments particularly in the United States China and the European Union National AI strategies emphasize talent cultivation data infrastructure and regulatory frameworks as levers of long term competitiveness Military applications of AI
ranging from autonomous drones to decision support systems in command and control illustrate the dual use nature of the technology raising fears of an AI arms race and the destabilizing effects of lethal autonomous weapons International norms and treaties lag behind
technological reality leaving a vacuum in which unilateral actions could have global consequences Advocates for AI diplomacy urge the establishment of guardrails transparency measures and verification mechanisms akin to nuclear arms control to prevent escalation and misuse
Culturally the spread of AI reshapes how societies create consume and value information In entertainment recommendation algorithms dictate much of what audiences watch read and listen to shaping tastes and market dynamics Streaming platforms and social media companies
leverage AI to maximize engagement but these optimizations can also foster filter bubbles polarization and addictive behaviors In the art world AI generated works challenge traditional notions of authorship authenticity and aesthetic merit sparking new movements while
provoking backlash from human artists Similarly journalism grapples with automated news generation and the need to distinguish reliable reporting from AI synthesized narratives Cultural adaptation to AI will likely involve a negotiation between human creativity and algorithmic
 influence emphasizing critical thinking media literacy and a renewed focus on human connection"""

In [76]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [77]:
tokens = word_tokenize(document.lower())

In [78]:
vocab = {'<unk>':0}
for token in Counter(tokens).keys():
  if token  not in vocab:
    vocab[token] = len(vocab)

vocab

{'<unk>': 0,
 'artificial': 1,
 'intelligence': 2,
 'often': 3,
 'abbreviated': 4,
 'as': 5,
 'ai': 6,
 'is': 7,
 'the': 8,
 'field': 9,
 'of': 10,
 'computer': 11,
 'science': 12,
 'that': 13,
 'focused': 14,
 'on': 15,
 'creating': 16,
 'machines': 17,
 'can': 18,
 'perform': 19,
 'tasks': 20,
 'normally': 21,
 'require': 22,
 'human': 23,
 'idea': 24,
 'think': 25,
 'reason': 26,
 'and': 27,
 'act': 28,
 'in': 29,
 'ways': 30,
 'similar': 31,
 'to': 32,
 'humans': 33,
 'has': 34,
 'fascinated': 35,
 'scientists': 36,
 'inventors': 37,
 'philosophers': 38,
 'for': 39,
 'centuries': 40,
 'modern': 41,
 'era': 42,
 'moved': 43,
 'from': 44,
 'a': 45,
 'theoretical': 46,
 'concept': 47,
 'into': 48,
 'practical': 49,
 'force': 50,
 'drives': 51,
 'many': 52,
 'aspects': 53,
 'our': 54,
 'lives': 55,
 'encompasses': 56,
 'multiple': 57,
 'branches': 58,
 'including': 59,
 'machine': 60,
 'learning': 61,
 'deep': 62,
 'natural': 63,
 'language': 64,
 'processing': 65,
 'vision': 66,
 'exp

In [79]:
input_sentences = document.split('\n')
input_sentences

['Artificial Intelligence often abbreviated as AI is the field of computer science that is focused on creating machines that can perform tasks that normally require human intelligence The idea of machines that can think reason and act in ways similar to ',
 'humans has fascinated scientists inventors and philosophers for centuries and in the modern era AI has moved from a theoretical concept into a practical force that drives many aspects of our lives Artificial Intelligence encompasses multiple branches including machine learning ',
 'deep learning natural language processing computer vision expert systems robotics and reinforcement learning and each branch contributes to the ability of machines to mimic human capabilities in some way Machine learning is perhaps the most widely used subset of AI where',
 ' algorithms are designed to improve their performance automatically by learning from data rather than through explicit programming Deep learning is a further specialization of machin

In [80]:
def text_to_indices(sentence, vocab):
  numerical_sentence = []
  for token in sentence:
    if token in vocab:
      numerical_sentence.append(vocab[token])
    else:
      numerical_sentence.append(vocab['<unk>'])

  return numerical_sentence

In [81]:
input_numerical_sentences = []
for sentence in input_sentences:
  input_numerical_sentences.append(text_to_indices(word_tokenize(sentence.lower()), vocab))

In [82]:
len(input_numerical_sentences)

162

In [83]:
training_sequence = []
for sentence in input_numerical_sentences:
  for i in range(1, len(sentence)):
    training_sequence.append(sentence[:i+1])

In [84]:
len(training_sequence)

5448

In [85]:
training_sequence[:5]

[[1, 2], [1, 2, 3], [1, 2, 3, 4], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5, 6]]

In [86]:
len_list = []
for sequence in training_sequence:
  len_list.append(len(sequence))

  max(len_list)

In [87]:
training_sequence[0]

[1, 2]

In [88]:
padded_traning_sequence = []
max_len = max(len_list)
for sequence in training_sequence:
  padding_length = max_len - len(sequence)
  padded_sequence = [0] * padding_length + sequence
  padded_traning_sequence.append(padded_sequence)
print(padded_traning_sequence)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 6], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [89]:
padded_traning_sequence = torch.tensor(padded_traning_sequence, dtype=torch.long)

In [90]:
padded_traning_sequence

tensor([[   0,    0,    0,  ...,    0,    1,    2],
        [   0,    0,    0,  ...,    1,    2,    3],
        [   0,    0,    0,  ...,    2,    3,    4],
        ...,
        [   0,    0,    0,  ..., 1771,  386,   15],
        [   0,    0,    0,  ...,  386,   15,   23],
        [   0,    0,    0,  ...,   15,   23, 1772]])

In [91]:
X = padded_traning_sequence[:,:-1]
y = padded_traning_sequence[:,-1]


In [92]:
X

tensor([[   0,    0,    0,  ...,    0,    0,    1],
        [   0,    0,    0,  ...,    0,    1,    2],
        [   0,    0,    0,  ...,    1,    2,    3],
        ...,
        [   0,    0,    0,  ...,   45, 1771,  386],
        [   0,    0,    0,  ..., 1771,  386,   15],
        [   0,    0,    0,  ...,  386,   15,   23]])

In [93]:
y

tensor([   2,    3,    4,  ...,   15,   23, 1772])

In [94]:
from os import XATTR_SIZE_MAX
class costumDataset(Dataset):
  def __init__(self,X ,y):
    self.X = X
    self.y = y

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

In [95]:
dataset = costumDataset(X,y)


In [96]:
len(dataset)

5448

In [97]:
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [98]:
for input , output in dataloader:
  print(input)
  print(output)
  break

tensor([[   0,    0,    0,  ...,   92, 1520, 1521],
        [   0,    0,    0,  ...,  543,   45,  280],
        [   0,    0,    0,  ...,    0,    0,  318],
        ...,
        [   0,    0,    0,  ...,   17,   32,   75],
        [   0,    0,    0,  ...,  654, 1274,   10],
        [   0,    0,    0,  ...,    6,  685,   27]])
tensor([ 173,  544,  468,  531,  419,  359,  359,  397,  748,  327, 1555,  162,
        1308,  726,  491,  373,   18, 1442,    7,  233,    8,   54,  639,   10,
         104,  171,  587,   11,    2,   23,  613,  222])


In [99]:
dataset[0]

(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
 tensor(2))

In [100]:
class LSTMModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, 100)
    self.lstm = nn.LSTM(100, 150, batch_first=True)
    self.fc = nn.Linear(150, vocab_size)

  def forward(self, x):
    embedded = self.embedding(x)
    intermediate_hidden_states, (final_hidden_state, final_cell_state) = self.lstm(embedded)
    output = self.fc(final_hidden_state.squeeze(0))
    return output

In [101]:
model = LSTMModel(len(vocab))

In [102]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [103]:
model.to(device)

LSTMModel(
  (embedding): Embedding(1773, 100)
  (lstm): LSTM(100, 150, batch_first=True)
  (fc): Linear(in_features=150, out_features=1773, bias=True)
)

In [104]:
epochs = 50
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [105]:

for epoch in range(epochs):
  total_loss = 0

  for batch_x, batch_y in dataloader:

    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    optimizer.zero_grad()

    output = model(batch_x)

    loss = criterion(output, batch_y)

    loss.backward()

    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch + 1}, Loss: {total_loss:.4f}")

Epoch: 1, Loss: 1189.1940
Epoch: 2, Loss: 1066.4675
Epoch: 3, Loss: 974.5382
Epoch: 4, Loss: 863.3641
Epoch: 5, Loss: 748.3674
Epoch: 6, Loss: 635.9238
Epoch: 7, Loss: 530.5374
Epoch: 8, Loss: 436.1165
Epoch: 9, Loss: 350.4786
Epoch: 10, Loss: 279.1675
Epoch: 11, Loss: 220.3834
Epoch: 12, Loss: 172.0159
Epoch: 13, Loss: 134.8502
Epoch: 14, Loss: 105.4990
Epoch: 15, Loss: 82.3773
Epoch: 16, Loss: 65.5692
Epoch: 17, Loss: 52.1710
Epoch: 18, Loss: 42.3105
Epoch: 19, Loss: 35.0257
Epoch: 20, Loss: 29.4679
Epoch: 21, Loss: 25.0928
Epoch: 22, Loss: 21.7996
Epoch: 23, Loss: 19.0675
Epoch: 24, Loss: 16.9753
Epoch: 25, Loss: 15.5818
Epoch: 26, Loss: 13.8218
Epoch: 27, Loss: 12.5996
Epoch: 28, Loss: 11.6599
Epoch: 29, Loss: 10.7297
Epoch: 30, Loss: 10.0502
Epoch: 31, Loss: 9.3195
Epoch: 32, Loss: 8.8077
Epoch: 33, Loss: 8.4090
Epoch: 34, Loss: 7.9577
Epoch: 35, Loss: 7.4706
Epoch: 36, Loss: 7.3511
Epoch: 37, Loss: 7.2170
Epoch: 38, Loss: 6.6161
Epoch: 39, Loss: 6.3958
Epoch: 40, Loss: 6.1291
Epo

In [106]:
def prediction(model, vocab, text):

  # tokenize
  tokenized_text = word_tokenize(text.lower())

  # text -> numerical indices
  numerical_text = text_to_indices(tokenized_text, vocab)

  # padding
  padded_text = torch.tensor([0] * (max_len - len(numerical_text)) + numerical_text, dtype=torch.long).unsqueeze(0)

  # send to model
  padded_text = padded_text.to(device) # Move input tensor to the same device as the model
  output = model(padded_text)

  # predicted index
  value, index = torch.max(output, dim=1)

  # merge with text
  return text + " " + list(vocab.keys())[index]

In [107]:
prediction(model, vocab, "The course follows a monthly")

'The course follows a monthly to'

In [112]:
import time

num_tokens = 20
input_text = "AI"

for i in range(num_tokens):
  output_text = prediction(model, vocab, input_text)
  print(output_text)
  input_text = output_text
  time.sleep(0.5)

AI research
AI research continues
AI research continues to
AI research continues to explore
AI research continues to explore frontiers
AI research continues to explore frontiers such
AI research continues to explore frontiers such as
AI research continues to explore frontiers such as artificial
AI research continues to explore frontiers such as artificial general
AI research continues to explore frontiers such as artificial general intelligence
AI research continues to explore frontiers such as artificial general intelligence agi
AI research continues to explore frontiers such as artificial general intelligence agi which
AI research continues to explore frontiers such as artificial general intelligence agi which refers
AI research continues to explore frontiers such as artificial general intelligence agi which refers to
AI research continues to explore frontiers such as artificial general intelligence agi which refers to systems
AI research continues to explore frontiers such as artifi

In [111]:
def calculate_accuracy(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_x, batch_y in dataloader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)


            outputs = model(batch_x)


            _, predicted = torch.max(outputs, dim=1)


            correct += (predicted == batch_y).sum().item()
            total += batch_y.size(0)

    accuracy = correct / total * 100
    return accuracy

# Compute accuracy
accuracy = calculate_accuracy(model, dataloader, device)
print(f"Model Accuracy: {accuracy:.2f}%")

Model Accuracy: 99.27%
